# test-tooling.ipynb

Test the API implementation of tooling

In [1]:
import os, sys
from typing import Optional, List, Dict
sys.path.append(os.path.join(".."))
from llms_wrapper.llms import LLMS, toolnames2funcs, get_func_by_name
from llms_wrapper.config import update_llm_config

In [2]:
config = dict(
    llms=[
        # OpenAI
        # https://platform.openai.com/docs/models
        dict(llm="openai/gpt-4o"),
        dict(llm="openai/gpt-4o-mini"),
        # dict(llm="openai/o1"),        # restricted
        # dict(llm="openai/o1-mini"),   # restricted
        # Google Gemini
        # https://ai.google.dev/gemini-api/docs/models/gemini
        dict(llm="gemini/gemini-2.0-flash-exp"),
        dict(llm="gemini/gemini-1.5-flash"),
        dict(llm="gemini/gemini-1.5-pro"),
        # Anthropic
        # https://docs.anthropic.com/en/docs/about-claude/models
        dict(llm="anthropic/claude-3-5-sonnet-20240620"),
        dict(llm="anthropic/claude-3-opus-20240229"),
        # Mistral
        # https://docs.mistral.ai/getting-started/models/models_overview/
        dict(llm="mistral/mistral-large-latest"),
        # XAI
        # dict(llm="xai/grok-2"),     # not mapped by litellm yet?
        dict(llm="xai/grok-beta"),
        # Groq
        # https://console.groq.com/docs/models
        dict(llm="groq/llama3-70b-8192"),
        dict(llm="groq/llama-3.3-70b-versatile"),
        # Deepseek
        # https://api-docs.deepseek.com/quick_start/pricing
        dict(llm="deepseek/deepseek-chat"),
    ],
    providers = dict(
        openai = dict(api_key_env="MY_OPENAI_API_KEY"),
        gemini = dict(api_key_env="MY_GEMINI_API_KEY"),
        anthropic = dict(api_key_env="MY_ANTHROPIC_API_KEY"),
        mistral = dict(api_key_env="MY_MISTRAL_API_KEY"),
        xai = dict(api_key_env="MY_XAI_API_KEY"),    
        groq = dict(api_key_env="MY_GROQ_API_KEY"),
        deepseek = dict(api_key_env="MY_DEEPSEEK_API_KEY"),
    )
)
config = update_llm_config(config)
llms = LLMS(config)
llms.list_aliases()

['openai/gpt-4o',
 'openai/gpt-4o-mini',
 'gemini/gemini-2.0-flash-exp',
 'gemini/gemini-1.5-flash',
 'gemini/gemini-1.5-pro',
 'anthropic/claude-3-5-sonnet-20240620',
 'anthropic/claude-3-opus-20240229',
 'mistral/mistral-large-latest',
 'xai/grok-beta',
 'groq/llama3-70b-8192',
 'groq/llama-3.3-70b-versatile',
 'deepseek/deepseek-chat']

## Define a bunch of fake tools

We simulate a situation where tooling is used to query a person/employee database with regard to various criteria. 

* Persons have an id,  first and last name, department id,  home city, age, date since employed
* Departments have a department id, department name and a department head identified by a person id
* Projects have an id, a project name and a person id list field of persons working on the project

Tools:
* get_pers(id=None, first=None, last=None, depid=None, city=None)
* get_dep(id=None, headid=None)
* get_proj(id=None, personid=None)



In [44]:
PERSONS = [
    dict(id=1, first="John", last="Doe", depid=1, city="New York"),
    dict(id=2, first="Jane", last="Smith", depid=2, city="Los Angeles"),
    dict(id=3, first="Alice", last="Johnson", depid=1, city="Chicago"),
    dict(id=4, first="Bob", last="Brown", depid=3, city="Houston"),
    dict(id=5, first="Charlie", last="Davis", depid=2, city="Phoenix"),
    dict(id=6, first="David", last="Wilson", depid=3, city="Philadelphia"),
    dict(id=7, first="Jane", last="Garcia", depid=1, city="San Antonio"),
    dict(id=8, first="Frank", last="Martinez", depid=2, city="San Diego"),
    dict(id=9, first="Grace", last="Lopez", depid=3, city="Houston"),
    dict(id=10, first="Heidi", last="Gonzalez", depid=1, city="San Jose"),
    dict(id=11, first="Ivan", last="Hernandez", depid=2, city="Austin"),
    dict(id=12, first="Jane", last="Clark", depid=3, city="Jacksonville"),
    dict(id=13, first="Kevin", last="Rodriguez", depid=1, city="Fort Worth"),
    dict(id=14, first="Laura", last="Lewis", depid=2, city="Columbus"),
]

DEPARTMENTS = [
    dict(id=1, name="HR", headid=1),
    dict(id=2, name="Engineering", headid=5),
    dict(id=3, name="Marketing", headid=3),
]


PROJECTS = [
    dict(id=1, name="Project A", people=[1, 2]),
    dict(id=2, name="Project B", people=[3, 4, 5]),
    dict(id=3, name="Project C", people=[5, 6, 7, 8]),
    dict(id=4, name="Project D", people=[1, 6, 5, 9, 10]),
    dict(id=5, name="Project E", people=[2, 3, 4, 11]),
]
 
def get_pers(id: Optional[int] = None, first: Optional[str] = None, last: Optional[str] = None, depid: Optional[int] = None, city: Optional[str] = None) -> List[Dict[str, any]]:
    """
    Get a list of person objects matching all the given criteria. 
    
    If no criteria is given, return all person objects. If no person matches the criteria, return an empty list.
    Note that if the ID is given, all other criteria are ignored.

    :param id: The ID of the person to return. Since the ID is unique, this will return a list with one element or an empty list.
    :param first: The first name of the person or persons to return.
    :param last: The last name of the person or persons to return.
    :param depid: The department ID of the person or persons to return.
    :param city: The city of the person or persons to return.
    :return: A list of person objects matching the given criteria.
    :rtype: List[Dict[str, any]]
    """
    if id is None and first is None and last is None and depid is None and city is None:
        return PERSONS
    if id is not None:
        return [p for p in PERSONS if p["id"] == id]
    retset = set()  # collect the ids of the persons that match the criteria
    if first is not None:        
        retset.update(p["id"] for p in PERSONS if p["first"] == first)
    if last is not None:
        retset.update(p["id"] for p in PERSONS if p["last"] == last)
    if depid is not None:
        retset.update(p["id"] for p in PERSONS if p["depid"] == depid)
    if city is not None:
        retset.update(p["id"] for p in PERSONS if p["city"] == city)
    # now return the persons that match the ids in the retset        
    return list(p for p in PERSONS if p["id"] in retset)

def get_dep(id: Optional[int] = None, name: Optional[str] = None, headid: Optional[int] = None) -> List[Dict[str, any]]:
    """
    Get a list of department objects matching all the given criteria. 
    
    If no criteria is given, return all department objects. If no department matches the criteria, return an empty list.
    Note that if the ID is given, all other criteria are ignored.

    :param id: The ID of the department to return. Since the ID is unique, this will return a list with one element or an empty list.
    :param name: The name of the department to return.
    :return: A list of department objects matching the given criteria.
    :rtype: List[Dict[str, any]]
    """
    if id is None and name is None and headid is None:
        return DEPARTMENTS
    if id is not None:
        return [d for d in DEPARTMENTS if d["id"] == id]
    retset = set()
    if name is not None:
        retset.update(d["id"] for d in DEPARTMENTS if d["name"] == name)
    if headid is not None:
        retset.update(d["id"] for d in DEPARTMENTS if d["headid"] == headid)
    return list(d for d in DEPARTMENTS if d["id"] in retset)

def get_proj(id: Optional[int] = None, name: Optional[str] = None, people: Optional[List[int]] = None) -> List[Dict[str, any]]:
    """
    Get a list of project objects matching all the given criteria.
    If no criteria is given, return all project objects. If no project matches the criteria, return an empty list.
    Note that if the ID is given, all other criteria are ignored.
    :param id: The ID of the project to return. Since the ID is unique, this will return a list with one element or an empty list.
    :param name: The name of the project to return.
    :param people: A list of people, return all projects where all the people in this list are involved.
    :return: A list of project objects matching the given criteria.
    :rtype: List[Dict[str, any]]
    """
    if id is None and name is None and people is None:
        return PROJECTS
    if id is not None:
        return [p for p in PROJECTS if p["id"] == id]
    retset = set()
    if name is not None:
        retset.update(p["id"] for p in PROJECTS if p["name"] == name)
    if people is not None:
        # for each project check if the people ids in the people parameter list are all in the project's people list
        for p in PROJECTS:
            if all(pid in p["people"] for pid in people):
                retset.add(p["id"])
    return list(p for p in PROJECTS if p["id"] in retset)

In [43]:
get_proj()

[]

In [40]:
QUERIES = [
    "What is the name of the department John Doe works in?",
    "What is the name of the department that has Alice Johnson as head?",
    "Who is the head of the department in which Jane Smith works?",
    "In which department does Jane work?", 
    "Which projects have John Doe as a member?",
    "Which projects have more than 2 people?",
    "Which persons work in three projects?",
    "Which persons work both in the same department and come from the same city?",
    "Which persons work both on the same project and come from the same city?",
]

## Tooling setup 

In [5]:
tooling = LLMS.make_tooling([get_dep, get_pers, get_proj])
tooling

[{'type': 'function',
  'function': {'name': 'get_dep',
   'description': "Get a list of department objects matching all the given criteria. \n\nIf no criteria is given, return all department objects. If no department matches the criteria, return an empty list.\nNote that if the ID is given, all other criteria are ignored.\n\nThe function returns: A list of department objects matching the given criteria.\n\nThe return type is: {'type': 'array', 'items': {'type': 'object', 'additionalProperties': {'type': 'string'}}}",
   'parameters': {'type': 'object',
    'properties': {'id': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
      'description': 'The ID of the department to return. Since the ID is unique, this will return a list with one element or an empty list.',
      'default': None},
     'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
      'description': 'The name of the department to return.',
      'default': None},
     'headid': {'anyOf': [{'type': 'integer'}, {

In [20]:
import warnings
from IPython.display import display, Markdown
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

def answer_query(query, llm="openai/gpt-4o", debug=False, show=True):
    # NOTE if show is True, the query and answer are shown in the notebook and nothing is returned, otherwise
    # the complete return value is returned
    md = ""
    if show:
        md += f"Query: {query}\n\n"
    msgs = LLMS.make_messages(query)
    ret = llms.query(llm, msgs, tools=tooling, return_cost=True, debug=debug)
    cost = ret["cost"]
    if show:
        if ret["error"]:
            md += f"**!!!Error**: {ret['error']}\n\nCost: {cost}"
        else:
            md += f"Answer: {ret['answer']}\n\nCost: {cost}"
        display(Markdown(md))
    else:
        return ret
    
    

In [22]:
answer_query(QUERIES[0], llm="openai/gpt-4o", debug=False, show=True)

Query: What is the name of the department John Doe works in?

Answer: John Doe works in the HR department.

Cost: 0.005665

In [21]:
answer_query(QUERIES[1], llm="openai/gpt-4o", debug=False, show=True)

Query: What is the name of the department that has Alice Johnson as head?

Answer: The department that has Alice Johnson as head is the Marketing department.

Cost: 0.0057375

In [23]:
answer_query(QUERIES[2], llm="openai/gpt-4o", debug=False, show=True)

Query: Who is the head of the department in which Jane Smith works?

Answer: The head of the department in which Jane Smith works is Charlie Davis.

Cost: 0.0083875

In [24]:
answer_query(QUERIES[3], llm="openai/gpt-4o", debug=False, show=True)

Query: In which department does Jane work?

Answer: There are multiple people named Jane:

1. Jane Smith works in the Engineering department.
2. Jane Garcia works in the HR department.
3. Jane Clark works in the Marketing department.

Cost: 0.006835000000000001

In [41]:
answer_query(QUERIES[4], llm="openai/gpt-4o", debug=False, show=True)

Query: Which projects have John Doe as a member?

Answer: John Doe is a member of the following projects:

1. Project A
2. Project D

Cost: 0.005840000000000001

In [45]:
answer_query(QUERIES[5], llm="openai/gpt-4o", debug=False, show=True)

Query: Which projects have more than 2 people?

Answer: The following projects have more than 2 people:

1. **Project B** - 3 people
2. **Project C** - 4 people
3. **Project D** - 5 people
4. **Project E** - 4 people

Cost: 0.00423

In [46]:
answer_query(QUERIES[6], llm="openai/gpt-4o", debug=False, show=True)

Query: Which persons work in three projects?

Answer: The persons who work in three projects are:

1. **Charlie Davis**
   - Projects: Project B, Project C, Project D
   - City: Phoenix

2. **David Wilson**
   - Projects: Project C, Project D
   - City: Philadelphia

3. **John Doe**
   - Projects: Project A, Project D
   - City: New York

4. **Jane Smith**
   - Projects: Project A, Project E
   - City: Los Angeles

These individuals are involved in multiple projects, with Charlie Davis being a part of exactly three different projects and holding the condition true as requested.

Cost: 0.0082675

In [47]:
answer_query(QUERIES[7], llm="openai/gpt-4o", debug=False, show=True)

Query: Which persons work both in the same department and come from the same city?

Answer: Based on the available data, the following persons work in the same department and come from the same city:

- Bob Brown and Grace Lopez both work in Department 3 and are from Houston.

Cost: 0.004922500000000001

In [48]:
answer_query(QUERIES[8], llm="openai/gpt-4o", debug=False, show=True)

Query: Which persons work both on the same project and come from the same city?

Answer: Here are the persons who work on the same project and come from the same city:

1. **Project A:** 
   - John Doe from New York
   - Jane Smith from Los Angeles

2. **Project B:** 
   - Alice Johnson from Chicago
   - Bob Brown from Houston

3. **Project C:** 
   - Charlie Davis from Phoenix
   - David Wilson from Philadelphia

4. **Project D:**
   - Grace Lopez from Houston
   - Heidi Gonzalez from San Jose

5. **Project E:** 
   - Each person from a different city

It seems no persons on the same project are from the same city based on the given data.

Cost: 0.010327500000000002